## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install pyspark==3.4.0 pandas numpy scikit-learn openjdk-11-jdk-headless -q
print("Dependencies installed successfully!")

## Step 2: Set Up Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Create Spark session
spark = SparkSession.builder \
    .appName("SmartTollSystem") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print(f"Spark Session created successfully!")
print(f"Version: {spark.version}")
print(f"Partitions: {spark.sparkContext.defaultParallelism}")

## Step 3: Generate Sample Toll Transaction Data

In [ ]:
def generate_toll_data(num_records=5000):
    """
    Generate realistic sample toll transaction data
    """
    print(f"[DataGen] Generating {num_records} toll transaction records...")
    
    np.random.seed(42)
    random.seed(42)
    
    # Time range
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2023, 12, 31)
    
    # Parameters
    plazas = [1, 2, 3, 4]
    lanes = [1, 2, 3, 4]
    vehicle_types = ['bike', 'car', 'truck', 'bus', 'heavy_vehicle']
    payment_modes = ['wallet', 'cash', 'upi']
    
    # Toll amounts by vehicle type
    toll_amounts = {
        'bike': [20, 40],
        'car': [40, 80],
        'truck': [120, 200],
        'bus': [80, 150],
        'heavy_vehicle': [180, 250]
    }
    
    data = []
    current_date = start_date
    
    while current_date <= end_date and len(data) < num_records:
        hour = current_date.hour
        day_of_week = current_date.weekday()
        
        # Determine number of vehicles in this hour
        base_vehicles = 60
        
        # Peak hours boost
        if (7 <= hour < 10) or (17 <= hour < 20):
            base_vehicles = 150
        
        # Weekend lighter
        if day_of_week >= 5:
            base_vehicles = int(base_vehicles * 0.8)
        
        # Night is light
        if hour < 6 or hour > 22:
            base_vehicles = int(base_vehicles * 0.3)
        
        num_vehicles = max(0, int(base_vehicles + np.random.normal(0, 20)))
        
        # Generate transactions
        for _ in range(num_vehicles):
            if len(data) >= num_records:
                break
            
            vehicle_type = random.choice(vehicle_types)
            toll_amount = random.uniform(*toll_amounts[vehicle_type])
            status = 'completed' if random.random() > 0.02 else 'failed'
            
            record = {
                'txn_id': len(data) + 1,
                'vehicle_id': random.randint(1, 500),
                'plaza_id': random.choice(plazas),
                'lane_no': random.choice(lanes),
                'timestamp': current_date + timedelta(minutes=random.randint(0, 59)),
                'amount': round(toll_amount, 2),
                'vehicle_type': vehicle_type,
                'payment_mode': random.choice(payment_modes),
                'status': status
            }
            data.append(record)
        
        current_date += timedelta(hours=1)
    
    df = pd.DataFrame(data)
    print(f"[DataGen] Generated {len(df)} records")
    print(f"[DataGen] Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    
    return df

# Generate data
pandas_df = generate_toll_data(num_records=5000)
print(f"\n[Data] First few records:")
print(pandas_df.head(10))

## Step 4: Convert to Spark DataFrame and Perform Analytics

In [ ]:
# Convert Pandas to Spark DataFrame
spark_df = spark.createDataFrame(pandas_df)

print("[Spark] DataFrame created")
spark_df.printSchema()
print(f"[Spark] Total records: {spark_df.count()}")

### Analytics 1: Hourly Vehicle Aggregation

In [ ]:
# Hourly aggregation
hourly = spark_df.withColumn(
    "timestamp",
    F.col("timestamp").cast(TimestampType())
).withColumn(
    "date",
    F.date_trunc("day", F.col("timestamp"))
).withColumn(
    "hour",
    F.hour(F.col("timestamp"))
).groupBy(
    F.col("plaza_id"),
    F.col("date"),
    F.col("hour")
).agg(
    F.count("*").alias("vehicle_count"),
    F.sum("amount").alias("total_revenue")
).orderBy(
    F.col("plaza_id"),
    F.col("date"),
    F.col("hour")
)

print("[Analytics] Hourly Vehicle Aggregation (Sample):")
hourly.limit(15).show()

### Analytics 2: Revenue Per Plaza

In [ ]:
# Revenue per plaza
revenue_per_plaza = spark_df.groupBy(F.col("plaza_id")).agg(
    F.count("*").alias("total_vehicles"),
    F.sum("amount").alias("total_revenue"),
    F.avg("amount").alias("avg_toll_amount")
).orderBy(F.desc("total_revenue"))

print("[Analytics] Revenue Per Plaza:")
revenue_per_plaza.show()

# Convert to Pandas for visualization
revenue_pandas = revenue_per_plaza.toPandas()
print("\n[Pandas] Revenue Data:")
print(revenue_pandas)

### Analytics 3: Vehicle Type Distribution

In [ ]:
# Vehicle type aggregation
vehicle_type_stats = spark_df.groupBy(F.col("vehicle_type")).agg(
    F.count("*").alias("vehicle_count"),
    F.sum("amount").alias("total_revenue"),
    F.avg("amount").alias("avg_toll_amount")
).orderBy(F.desc("vehicle_count"))

print("[Analytics] Vehicle Type Distribution:")
vehicle_type_stats.show()

# Pie chart
import matplotlib.pyplot as plt

vehicle_pandas = vehicle_type_stats.toPandas()
plt.figure(figsize=(10, 6))
plt.pie(vehicle_pandas['vehicle_count'], labels=vehicle_pandas['vehicle_type'], autopct='%1.1f%%')
plt.title('Vehicle Count Distribution by Type')
plt.show()

### Analytics 4: Peak Hours Detection

In [ ]:
# Peak hours
peak_hours = spark_df.withColumn(
    "timestamp",
    F.col("timestamp").cast(TimestampType())
).withColumn(
    "hour",
    F.hour(F.col("timestamp"))
).groupBy(
    F.col("hour")
).agg(
    F.count("*").alias("vehicle_count"),
    F.sum("amount").alias("total_revenue")
).orderBy(F.desc("vehicle_count")).limit(5)

print("[Analytics] Top 5 Peak Hours:")
peak_hours.show()

# Bar chart
peak_pandas = peak_hours.toPandas()
plt.figure(figsize=(12, 6))
plt.bar(peak_pandas['hour'], peak_pandas['vehicle_count'], color='skyblue')
plt.xlabel('Hour of Day')
plt.ylabel('Vehicle Count')
plt.title('Peak Traffic Hours')
plt.grid(axis='y')
plt.show()

### Analytics 5: Traffic Log Creation (for ML)

In [ ]:
# Create traffic log with traffic level classification
traffic_log = spark_df.withColumn(
    "timestamp",
    F.col("timestamp").cast(TimestampType())
).withColumn(
    "date",
    F.date_trunc("day", F.col("timestamp"))
).withColumn(
    "hour",
    F.hour(F.col("timestamp"))
).groupBy(
    F.col("plaza_id"),
    F.col("date"),
    F.col("hour")
).agg(
    F.count("*").alias("vehicle_count"),
    F.sum("amount").alias("total_revenue")
)

# Classify traffic level
p50 = traffic_log.approxQuantile("vehicle_count", [0.5], 0.01)[0]
p75 = traffic_log.approxQuantile("vehicle_count", [0.75], 0.01)[0]

traffic_log = traffic_log.withColumn(
    "traffic_level",
    F.when(F.col("vehicle_count") > p75, "high") \
        .when(F.col("vehicle_count") > p50, "normal") \
        .otherwise("low")
)

print(f"[Analytics] Traffic Log Statistics:")
print(f"  50th percentile (median): {p50:.0f} vehicles")
print(f"  75th percentile: {p75:.0f} vehicles")
print(f"\n[Analytics] Sample Traffic Log:")
traffic_log.limit(20).show()

# Convert to Pandas for ML
traffic_log_pandas = traffic_log.toPandas()
print(f"\n[Data] Total traffic log records: {len(traffic_log_pandas)}")

## Step 5: Machine Learning - Train Traffic Prediction Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print("\n[ML] Preparing data for model training...")

# Prepare data
ml_data = spark_df.withColumn(
    "timestamp",
    F.col("timestamp").cast(TimestampType())
).withColumn(
    "hour",
    F.hour(F.col("timestamp"))
).withColumn(
    "day_of_week",
    F.dayofweek(F.col("timestamp"))
).select(
    "plaza_id", "hour", "day_of_week", "amount"
)

# Convert to Pandas
ml_pandas = ml_data.toPandas()

# Add features
ml_pandas['is_peak_hour'] = (
    ((ml_pandas['hour'] >= 7) & (ml_pandas['hour'] < 10)) | 
    ((ml_pandas['hour'] >= 17) & (ml_pandas['hour'] < 20))
).astype(int)

# Group by hour to get vehicle count as target
hourly_data = ml_pandas.groupby(['plaza_id', 'hour', 'day_of_week', 'is_peak_hour']).size().reset_index(name='vehicle_count')

print(f"[ML] Created {len(hourly_data)} hourly records for training")

# Features and target
X = hourly_data[['plaza_id', 'hour', 'day_of_week', 'is_peak_hour']].values
y = hourly_data['vehicle_count'].values

print(f"[ML] Feature shape: {X.shape}, Target shape: {y.shape}")

### Train Random Forest Model

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Random Forest model
print("\n[ML] Training Random Forest model...")
model = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train)

# Predictions
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

# Evaluation metrics
print("\n[ML] ===== TRAINING RESULTS =====")
print(f"Train R² Score: {r2_score(y_train, y_pred_train):.4f}")
print(f"Test R² Score:  {r2_score(y_test, y_pred_test):.4f}")
print(f"Train RMSE:     {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")
print(f"Test RMSE:      {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"Train MAE:      {mean_absolute_error(y_train, y_pred_train):.2f}")
print(f"Test MAE:       {mean_absolute_error(y_test, y_pred_test):.2f}")

# Feature importance
feature_names = ['plaza_id', 'hour', 'day_of_week', 'is_peak_hour']
print("\n[ML] Feature Importance:")
for name, importance in zip(feature_names, model.feature_importances_):
    print(f"  {name}: {importance:.4f}")

### Visualize Model Performance

In [ ]:
# Feature importance plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Feature importance
axes[0, 0].barh(feature_names, model.feature_importances_, color='skyblue')
axes[0, 0].set_xlabel('Importance')
axes[0, 0].set_title('Feature Importance')
axes[0, 0].grid(axis='x')

# Actual vs Predicted (Test set)
axes[0, 1].scatter(y_test, y_pred_test, alpha=0.6)
axes[0, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0, 1].set_xlabel('Actual Vehicle Count')
axes[0, 1].set_ylabel('Predicted Vehicle Count')
axes[0, 1].set_title('Actual vs Predicted (Test Set)')
axes[0, 1].grid(True)

# Residuals
residuals = y_test - y_pred_test
axes[1, 0].scatter(y_pred_test, residuals, alpha=0.6)
axes[1, 0].axhline(y=0, color='r', linestyle='--')
axes[1, 0].set_xlabel('Predicted Values')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].set_title('Residual Plot')
axes[1, 0].grid(True)

# Error distribution
axes[1, 1].hist(residuals, bins=20, edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Residual Value')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Residual Distribution')
axes[1, 1].grid(axis='y')

plt.tight_layout()
plt.show()

print("[ML] Model visualization complete")

## Step 6: Make Predictions

In [ ]:
# Make predictions for future hours
print("\n[Prediction] Predicting traffic for next 6 hours...\n")

current_hour = datetime.now().hour
current_day = datetime.now().weekday()
plaza_id = 1

predictions = []
for i in range(1, 7):
    hour = (current_hour + i) % 24
    is_peak = 1 if (7 <= hour < 10) or (17 <= hour < 20) else 0
    
    # Create feature array
    features = np.array([[plaza_id, hour, current_day, is_peak]])
    features_scaled = scaler.transform(features)
    
    # Predict
    predicted_vehicles = max(0, int(model.predict(features_scaled)[0]))
    
    # Determine traffic level
    if predicted_vehicles > p75:
        traffic_level = 'HIGH'
    elif predicted_vehicles > p50:
        traffic_level = 'NORMAL'
    else:
        traffic_level = 'LOW'
    
    predictions.append({
        'Hour': f"{hour:02d}:00",
        'Predicted Vehicles': predicted_vehicles,
        'Traffic Level': traffic_level,
        'Confidence': '75%'
    })
    
    print(f"Hour {hour:02d}:00 - Predicted: {predicted_vehicles} vehicles - Level: {traffic_level}")

# Create DataFrame for visualization
pred_df = pd.DataFrame(predictions)
print("\n[Prediction] Summary:")
print(pred_df.to_string(index=False))

## Step 7: Summary & Results

In [ ]:
print("\n" + "="*70)
print("SMART TOLL MANAGEMENT SYSTEM - ANALYSIS SUMMARY")
print("="*70)

print("\n[DATA GENERATION]")
print(f"  Total Records: {len(pandas_df):,}")
print(f"  Time Period: {pandas_df['timestamp'].min()} to {pandas_df['timestamp'].max()}")
print(f"  Toll Plazas: 4")
print(f"  Total Revenue: Rs. {pandas_df['amount'].sum():.2f}")
print(f"  Average Toll: Rs. {pandas_df['amount'].mean():.2f}")

print("\n[SPARK ANALYTICS]")
print(f"  Hourly Records Generated: {len(traffic_log_pandas):,}")
print(f"  Peak Hour Vehicle Count: {traffic_log_pandas['vehicle_count'].max()}")
print(f"  Low Traffic Vehicle Count: {traffic_log_pandas['vehicle_count'].min()}")

print("\n[MACHINE LEARNING]")
print(f"  Model Type: Random Forest Regressor")
print(f"  Training Samples: {len(X_train):,}")
print(f"  Test Samples: {len(X_test):,}")
print(f"  Test R² Score: {r2_score(y_test, y_pred_test):.4f}")
print(f"  Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"  Test MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")

print("\n[KEY INSIGHTS]")
print(f"  Most Important Feature: {feature_names[np.argmax(model.feature_importances_)]}")
print(f"  Peak Hour Traffic: {p75:.0f}+ vehicles")
print(f"  Normal Hour Traffic: {p50:.0f}-{p75:.0f} vehicles")
print(f"  Low Traffic: <{p50:.0f} vehicles")

print("\n" + "="*70)
print("Analysis Complete! Ready for deployment.")
print("="*70)